
First, let's simulate some phenotypes (using your Phoenix data):

In [261]:
import scipy.io
import sksparse.cholmod
import scipy.sparse
import scipy.stats
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')


A=scipy.io.mmread("./A.mtx")


#Copy the population a few times to get more data
A=scipy.sparse.block_diag((A,A)*20)

Calculate the cholesky decomposition:

In [204]:
A_chol_obj=sksparse.cholmod.cholesky(A)
P=scipy.sparse.identity(A.shape[0]).tocsc()[A_chol_obj.P()[:,np.newaxis],np.arange(A.shape[0])[np.newaxis,:]]
A_chol=P.transpose()*A_chol_obj.L()

Check that I did this correctly:

In [205]:
np.max(np.max(A_chol*A_chol.transpose()-A,0),1).todense()

matrix([[2.22044605e-16]])

Let's simulate 10 phenotypes with heritability=75%:

In [252]:
phenos=["NA"]*10
heritability=0.75

In [265]:

for i in range(10):
    random.seed(10)
    phenos[i]=A_chol*np.random.normal(size=A.shape[0])*np.sqrt(heritability)+scipy.sparse.identity(A.shape[0])*np.random.normal(size=A.shape[0])*np.sqrt(1-heritability)

Shor T, et al. (2019) "Estimating variance components in population scale family trees" has a method for calculating the heritability of continuous traits. Let's see if it works:

In [254]:
def HE(mat_list, y, MQS=False, verbose=False, sim_num=100, compute_stderr=False, y2=None):
    # regress all covariates out of y
    #CTC = cov.T.dot(cov)
    y = y - np.mean(y)

    # standardize y
    y /= y.std()
    # assert np.isclose(y.mean(), 0)
    # assert np.isclose(y.var(), 1)
    
    if y2 is not None:
        y2 -= cov.dot(np.linalg.solve(CTC, cov.T.dot(y2)))
        y2 /= y2.std()
        y = np.concatenate((y, y2))
        
        for m_i, m in enumerate(mat_list):
            z = sparse.csr_matrix((m.shape[0], m.shape[0]))
            mz = sparse.hstack([m,z])
            zm = sparse.hstack([z,m])
            mat_list[m_i] = sparse.vstack([zm, mz]).tocsr()
            

    # construct S and q, without MQS
    K = len(mat_list)
    n = y.shape[0]
    if not MQS:
        q = np.zeros(K)
        S = np.zeros((K, K))
        for i, mat_i in enumerate(mat_list):
            if scipy.sparse.issparse(mat_i):
                # q[i] = ((mat_i.multiply(y)).T.tocsr().dot(y)).sum() - mat_i.diagonal().dot(y**2)
                q[i] = y.dot(mat_i.dot(y)) - mat_i.diagonal().dot(y ** 2)
            else:
                q[i] = ((mat_i * y).T.dot(y)).sum() - np.diag(mat_i).dot(y ** 2)
            for j, mat_j in enumerate(mat_list):
                if j > i: continue
                if scipy.sparse.issparse(mat_i):
                    S[i, j] = (mat_i.multiply(mat_j)).sum() - mat_i.diagonal().dot(mat_j.diagonal())
                else:
                    S[i, j] = np.einsum('ij,ij->', mat_i, mat_j) - np.diag(mat_i).dot(np.diag(mat_j))
                S[j, i] = S[i, j]

    # construct S and q with MQS (it's almost the same thing...)
    else:        
        q = np.zeros(K)
        S = np.zeros((K, K))
        for i, mat_i in enumerate(mat_list):
            q[i] = (y.dot(mat_i.dot(y)) - y.dot(y))  # / float(n-1)**2
            for j, mat_j in enumerate(mat_list):
                if j > i: continue
                S[i, j] = ((mat_i.multiply(mat_j)).sum() - (n - 1))  # / float(n-1)**2
                S[j, i] = S[i, j]

    # compute HE
    he_est = np.linalg.solve(S, q)

    if not compute_stderr:
        return he_est

    # compute H - the covariance matrix of y
    H = mat_list[0] * he_est[0]
    for mat_i, sigma2_i in zip(mat_list[1:], he_est[1:]):
        H += mat_i * he_est[i]
    H += scipy.sparse.eye(y.shape[0], format='csr') * (1.0 - he_est.sum())

    # compute HE sampling variance
    V_q = np.empty((K, K))
    for i, mat_i in enumerate(mat_list):
        if sim_num is None:
            HAi_min_I = H.dot(mat_i) - H
        for j, mat_i in enumerate(mat_list[:i + 1]):
            if sim_num is None:
                if j == i:
                    HAj_min_I = HAi_min_I
                else:
                    HAj_min_I = H.dot(mat_j) - H
                V_q[i, j] = 2 * (HAi_min_I.multiply(HAj_min_I)).sum()  # / float(n-1)**4
            else:
                # simulate vectors
                sim_y = np.random.randn(n, sim_num)
                Aj_minI_y = mat_j.dot(sim_y) - sim_y
                H_Aj_minI_y = H.dot(Aj_minI_y)
                Ai_min_I_H_Aj_minI_y = mat_i.dot(H_Aj_minI_y) - H_Aj_minI_y
                H_Ai_min_I_H_Aj_minI_y = H.dot(Ai_min_I_H_Aj_minI_y)
                V_q[i, j] = 2 * np.mean(np.einsum('ij,ij->j', sim_y, H_Ai_min_I_H_Aj_minI_y))

            V_q[j, i] = V_q[i, j]
    var_he_est = np.linalg.solve(S, np.linalg.solve(S, V_q).T).T

    return he_est, np.sqrt(np.diag(var_he_est))


In [266]:
heritability_esti=['NA']*10
for i,p in enumerate(phenos):
    heritability_esti[i]=HE(mat_list=[A],y=p)

In [267]:
np.mean(heritability_esti)

0.7240552773742708

This number is pretty close to 75%. I've tested this method a lot, and it works pretty well, atleast if you have sufficient data. 

But let's see what happens if we transform the data to binary traits (1% prevalence):


In [268]:
pre=0.01
phenos_binary=[(p>scipy.stats.norm.ppf(1-pre))*1.0 for p in phenos]

We use the conversion formula from Dempster and Lerner (1950):

In [269]:
heritability_esti_binary=['NA']*10
for i,p in enumerate(phenos_binary):
    t=scipy.stats.norm.pdf(scipy.stats.norm.ppf(pre))
    heritability_esti_binary[i]=HE(mat_list=[A],y=p)*pre*(1-pre)/t**2

In [270]:
np.mean(heritability_esti_binary)

1.8707642630435948

This result is totally off. You get a heritability well over 100%.

In your article (Villemereuil et al. 2013; https://besjournals.onlinelibrary.wiley.com/doi/full/10.1111/2041-210X.12011), you use a prevalence of 1/3, which seems to work a lot better.

In [274]:
pre=1/3
phenos_binary=[(p>scipy.stats.norm.ppf(1-pre))*1.0 for p in phenos]

In [275]:
heritability_esti_binary=['NA']*10
for i,p in enumerate(phenos_binary):
    t=scipy.stats.norm.pdf(scipy.stats.norm.ppf(pre))
    heritability_esti_binary[i]=HE(mat_list=[A],y=p)*pre*(1-pre)/t**2

In [276]:
np.mean(heritability_esti_binary)

0.7389644844058678